To classify movie reviews as positive or negative, based on the text content of the reviews. using IMDB dataset 

In [36]:
from keras.datasets import imdb
import numpy as np
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
import matplotlib.pyplot as py

################# 3.1 Loading the IMDB dataset #######################################

(train_data, train_labels),(test_data, test_labels) = imdb.load_data(num_words=10000)

# train_data[0]
# train_labels[0]

# max([max(sequence) for sequence in train_data]) ### max word index
# word_index = imdb.get_word_index() ### word_index is a dictionary mapping words to an integer index.
# reverse_word_index = dict ([(value, key) for (value, key) in word_index.items()]) ### Reverses it, 
# ### mapping integer indices to words
# decoded_review = ''.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]]) ### Decodes 
# ### the review. Note that the indices are offset by 3 because 0, 1, and 2 are reserved indices
# ### for “padding,” “start of sequence,” and “unknown".

############# 3.2 Encoding the integer sequences into a binary matrix #################

def vactorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension)) ### Creates an all-zero matrix of shape (len(sequences), dimension)
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1. ### Sets specific indices of results[i] to 1s
    return results
    
x_train = vactorize_sequences(train_data) ### Vectorized training data
x_test = vactorize_sequences(test_data) ### Vectorized test data

# x_train

############## Vectorize labels #################

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

# y_train

####################### 3.3 The model definition ######################################

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

####################### 3.4 Compiling the model ######################################

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

####################### 3.5,6 configuring custom Optimizer, Loss and metric functino #########

# model.compile(optimizer=optimizers.RMSprop(lr=0.001), 
#               loss=losses.binary_crossentropy, 
#               metrics=[metrics.binary_accuracy])    
              
####################### 3.7 setting up validation set ################################      

#### create validation set of 10000 sample from training dataset
# x_val = x_train[:10000]
# partial_x_train = x_train[10000:]
# y_val = y_train[:10000]
# partial_y_train = y_train[10000:]

####################### 3.8 train the model #########################################

# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# history = model.fit(partial_x_train, 
#                     partial_y_train, 
#                     epochs=20, 
#                     batch_size=512, 
#                     validation_data=(x_val, y_val))

### model.fit() returns a History object as member history, which is a dictionary containing
### four entries: one per metric that was being monitored during training and during validation

# history_dict = history.history
# history_dict.keys()

####################### 3.9 Plotting the training and validation loss ###############

# history_dict = history.history
# loss_values = history_dict['loss']
# val_loss_values = history_dict['val_loss']
# acc = history_dict['acc']
# epochs = range(1, len(acc)+1)
# py.plot(epochs, loss_values, 'bo', label='Training Loss')
# py.plot(epochs, val_loss_values, 'b', label='Validation Loss')
# py.title('Training and Validation Loss')
# py.xlabel('Epochs')
# py.ylabel('Loss')
# py.legend()
# py.show()

### the training loss decreases with every epoch

####################### 3.10 Plotting the training and validation Accuracy ###############

# py.clf()
# acc_values = history_dict['acc']
# val_acc_values = history_dict['val_acc']
# py.plot(epochs, acc_values, 'bo', label='Training acc')
# py.plot(epochs, val_acc_values, 'b', label='Validation acc')
# py.title('Training and Validation Accuracy')
# py.xlabel('Epochs')
# py.ylabel('Loss')
# py.legend()
# py.show()

### the training accuracy increases with every epoch

####################### 3.11 Retraining the model #########################################

model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)
results ### with 4 epoch, accuracy of 88% is achieved

model.predict(x_test) ### Using a trained network to generate predictions on new data

Epoch 1/4
25000/25000 [==============================] - 4s 177us/step - loss: 0.4476 - acc: 0.8173
Epoch 2/4
25000/25000 [==============================] - 3s 135us/step - loss: 0.2543 - acc: 0.9100 1s - loss:
Epoch 3/4
25000/25000 [==============================] - 3s 135us/step - loss: 0.1966 - acc: 0.9298
Epoch 4/4
25000/25000 [==============================] - 4s 172us/step


array([[0.22259216],
       [0.9997013 ],
       [0.83085567],
       ...,
       [0.1950918 ],
       [0.06340346],
       [0.5874053 ]], dtype=float32)